In [1]:
# 匯入套件
import os                 # 處理資料夾建立
import re                 # 處理正則表達式（用來找中文字）
import requests as req    # 發送網路請求
from bs4 import BeautifulSoup as bs  # 解析 HTML 網頁

# 建立儲存書籍的資料夾
folderPath = 'Project_Gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 用來存放書籍資料的清單
Project_Gutenberg = []

# 中文書籍列表的頁面
url = 'https://www.gutenberg.org/browse/languages/zh'
res = req.get(url)
soup = bs(res.text, 'lxml')

# 取得所有書籍連結
book_links = soup.select("li.pgdbetext > a[href]")

# 建立用來抓取中文的正則表達式
zh_pattern = re.compile(r"[\u4e00-\u9fff，。！？「」『』、；：──．．…《》\s]+")

# 下載成功的書籍數量
count = 0

# 開始逐本處理書籍
for a in book_links:
    title = a.text.strip()                     # 書名
    href = a['href']                           # 書籍詳細頁面的連結
    book_url = f"https://www.gutenberg.org{href}"  # 完整網址

    try:
        # 進入書籍詳細頁面
        book_res = req.get(book_url)
        book_soup = bs(book_res.text, 'lxml')

        # 找出 UTF-8 編碼的純文字檔案連結
        link_tag = book_soup.find("a", string=re.compile(r"Plain Text UTF-8"))
        if not link_tag:
            continue  # 如果找不到就跳過

        # 拼出純文字檔案的完整網址
        txt_url = "https://www.gutenberg.org" + link_tag["href"]

        # 下載純文字內容
        txt_res = req.get(txt_url)
        txt_res.encoding = "utf-8"  # 設定正確編碼

        # 把純文字中符合中文的部分抓出來
        zh_text = "".join(zh_pattern.findall(txt_res.text))

        # 儲存成 .txt 檔案
        filename = f"{folderPath}/{title}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(zh_text)

        count += 1
        print(f"Success!!! {count} {title}")  # 顯示成功訊息

        if count >= 300:
            break  # 如果超過 300 本就停止

    except Exception as e:
        print(f"跳過 {title}：{e}")  # 發生錯誤時顯示訊息

# 最後統計下載結果
print(f"總共儲存 {count} 本中文書")



Success!!! 1 豆棚閒話
Success!!! 2 戲中戲


KeyboardInterrupt: 